In [ ]:
# Get imports
import pickle
import numpy as np
import cv2
import glob
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
%matplotlib qt

In [ ]:
# import helper functions
import camera_calibrator
import distortion_correction
import image_binary_gradient
import perspective_transform
import detect_lane_pixels
import measure_curvature

In [ ]:
# Files to undistort
calibration1 = ('./camera_cal/calibration1.jpg')
straight_lines1 = ('./test_images/straight_lines1.jpg')
straight_lines2 = ('./test_images/straight_lines2.jpg')
test1 = ('./test_images/test1.jpg')
test2 = ('./test_images/test2.jpg')
test3 = ('./test_images/test3.jpg')
test4 = ('./test_images/test4.jpg')
test5 = ('./test_images/test5.jpg')
test6 = ('./test_images/test6.jpg')


In [ ]:
def pipeline(image):
    
    # Get matrix from calibration
    mtx, dist = camera_calibrator.get_calibration_matrix()
    
    
    # Smoothen the lanes using clahe
    LUV = image
    lab = cv2.cvtColor(LUV, cv2.COLOR_RGB2LUV)
    lab_planes = cv2.split(lab)
    gridsize = 3
    clahe = cv2.createCLAHE(clipLimit=2.0,tileGridSize=(gridsize,gridsize))
    lab_planes[0] = clahe.apply(lab_planes[0])
    lab = cv2.merge(lab_planes)
    image_original = cv2.cvtColor(lab, cv2.COLOR_LUV2RGB)


    
    # Undistoretd image 
    undist_img = distortion_correction.get_undistorted_image(image_original, mtx, dist, gray_scale=False)#image_original
    
    # Binary Image 
    
    color_binary, binary_image = image_binary_gradient.get_binary_image(undist_img)

    
    # Perspective trasform
    top_down = perspective_transform.get_transformed_perspective(binary_image)  
    
    # Warp the color zone back to original image space using inverse perspective matrix (Minv)
    color_zone_warp = measure_curvature.get_color_zone_warp(top_down)
    newwarp  = perspective_transform.get_original_perspective(color_zone_warp)

    original_img = cv2.addWeighted(image_original, 1, newwarp, 0.3, 0)#image_original
    result = measure_curvature.add_text(original_img, top_down)
    
    # If gray scale convert to triple channel format
    
    if len(result.shape) == 2:
        result = np.dstack((original_img,)*3)

    # If binary image, scale to full 8-bit values
    if np.max(result) <= 1:
        result *= 255
    
    return result

In [ ]:
img = lambda fname: mpimg.imread(fname)
result = pipeline(img(test4))
plt.imshow(result)

#### Test pipeline on videos

In [ ]:
# Import everything needed to edit/save/watch video clips
from moviepy.editor import VideoFileClip
from IPython.display import HTML

In [ ]:
# fname = 'project_video.mp4'
# output = 'videos_out/project_video_processed.mp4'

# clip1 = VideoFileClip(fname)
# clip = clip1.fl_image(pipeline)
# %time clip.write_videofile(output, audio=False)

In [ ]:
# # Challange video

# clip2 = VideoFileClip('challenge_video.mp4')
# clip_challenge = clip2.fl_image(pipeline)
# %time clip_challenge.write_videofile('videos_out/challenge_video_processed.mp4', audio=False)